# Demand Data Collection

## Electricity Zones by Regional Transmission Organization (RTO)

## MISO (Midcontinent Independent System Operator)
- **(0001)** Zone 1
- **(0004)** Zone 4
- **(0006)** Zone 6
- **(0027)** Zones 2 and 7
- **(0035)** Zones 3 and 5
- **(8910)** Zones 8, 9 and 10

## ISNE (ISO New England)
- **(4001)** Maine
- **(4002)** New Hampshire
- **(4003)** Vermont
- **(4004)** Connecticut
- **(4005)** Rhode Island
- **(4006)** Southeast Mass.
- **(4007)** Western/Central Mass.
- **(4008)** Northeast Mass.

## PNM (Public Service Company of New Mexico)
- **(ACMA)** City of Acoma Pueblo
- **(CYGA)** PNM-CYGA
- **(Frep)** Frep
- **(Jica)** Jicarilla Apache Nation
- **(KAFB)** KAFB
- **(KCEC)** KCEC
- **(LAC)** Los Alamos County
- **(NTUA)** Navajo Tribal Utility Authority
- **(PNM)** PNM System Firm Load
- **(TSGT)** TSGT

## PJM (PJM Interconnection)
- **(AE)** Atlantic Electric zone
- **(AEP)** American Electric Power zone
- **(AP)** Allegheny Power zone
- **(ATSI)** American Transmission Systems, Inc. zone
- **(BC)** Baltimore Gas null zone
- **(CE)** Commonwealth Edison zone
- **(DAY)** Dayton Power null zone
- **(DEOK)** Duke Energy Ohio/Kentucky zone
- **(DOM)** Dominion Virginia Power zone
- **(DPL)** Delmarva Power null zone
- **(DUQ)** Duquesne Lighting Company zone
- **(EKPC)** East Kentucky Power Cooperative zone
- **(JC)** Jersey Central Power null zone
- **(ME)** Metropolitan Edison zone
- **(PE)** PECO Energy zone
- **(PEP)** Potomac Electric Power zone
- **(PL)** Pennsylvania Power zone
- **(PN)** Pennsylvania Electric zone
- **(PS)** Public Service Electric & Gas of New Jersey zone
- **(RECO)** Rockland Electric (East) zone

## ERCO (ERCOT - Electric Reliability Council of Texas)
- **(COAS)** Coast
- **(EAST)** East
- **(FWES)** Far West
- **(NCEN)** North Central
- **(NRTH)** North
- **(SCEN)** South Central
- **(SOUT)** South
- **(WEST)** West

## SWPP (Southwest Power Pool)
- **(CSWS)** AEPW American Electric Power West
- **(EDE)** Empire District Electric Company
- **(GRDA)** Grand River Dam Authority
- **(INDN)** Independence Power null
- **(KACY)** Kansas City Board of Public Utilities
- **(KCPL)** Kansas City Power & Light
- **(LES)** Lincoln Electric System
- **(MPS)** KCP&L Greater Missouri Operations
- **(NPPD)** Nebraska Public Power District
- **(OKGE)** Oklahoma Gas and Electric Co.
- **(OPPD)** Omaha Public Power District
- **(SECI)** Sunflower Electric
- **(SPRM)** City of Springfield
- **(SPS)** Southwestern Public Service Company
- **(WAUE)** Western Area Power Upper Great Plains East
- **(WFEC)** Western Farmers Electric Cooperative
- **(WR)** Westar Energy

## CISO (California ISO)
- **(PGAE)** Pacific Gas and Electric
- **(SCE)** Southern California Edison
- **(SDGE)** San Diego Gas and Electric
- **(VEA)** Valley Electric Association

## NYIS (New York Independent System Operator)
- **(ZONA)** West
- **(ZONB)** Genesee
- **(ZONC)** Central
- **(ZOND)** North
- **(ZONE)** Mohawk Valley
- **(ZONF)** Capital
- **(ZONG)** Hudson Valley
- **(ZONH)** Millwood
- **(ZONI)** Dunwoodie
- **(ZONJ)** New York City
- **(ZONK)** Long Island


In [1]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
    
    
DEMAND_API_KEY = "f8tGzRmnyw6dJyy1PyS49REmg1qrT2isvVi8i9mt"
WEATHER_API_KEY = '820479673a8444f69ac162421242809'

In [2]:
def split_dates_six_months(start_date, end_date):
    start = datetime.strptime(start_date, "%d-%m-%Y")
    end = datetime.strptime(end_date, "%d-%m-%Y")
    
    date_ranges = []
    current_date = start
    
    while current_date <= end:
        next_six_months = current_date + relativedelta(months=6)
        last_day_of_six_months = next_six_months - relativedelta(days=1)
        
        end_of_range = min(last_day_of_six_months, end)
        date_ranges.append([current_date.strftime("%Y-%m-%d"), end_of_range.strftime("%Y-%m-%d")])
        
        current_date = next_six_months
    
    return date_ranges


def get_demand_data(subba, start_date, end_date):
    print("Getting demand data")
    print(start_date, end_date)
    
    start_date = datetime.strptime(start_date, "%d-%m-%Y").strftime("%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%d-%m-%Y").strftime("%Y-%m-%d")

    demand_url = "https://api.eia.gov/v2/electricity/rto/region-sub-ba-data/data/?api_key={0}&data[]=value&facets[subba][]={1}&start={2}&end={3}".format(DEMAND_API_KEY, subba, start_date, end_date)

    response = requests.get(demand_url)

    if response.status_code == 200:
        json_data = response.json()
        df_demand = pd.DataFrame(json_data["response"]["data"])
        df_demand = df_demand.drop(columns=['subba', 'parent', 'parent-name'])
        return df_demand

    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return ""

subba = "COAS" # texas ERCO COAS
start_date = "01-01-2019"
end_date = "31-12-2019"

df_demand = get_demand_data(subba, start_date, end_date)


Getting demand data
01-01-2019 31-12-2019


In [3]:
df_demand.head()

,period,subba-name,value,value-units
0,2019-12-31T00,ERCO - Coast,10570,megawatthours
1,2019-12-30T23,ERCO - Coast,10235,megawatthours
2,2019-12-30T22,ERCO - Coast,10270,megawatthours
3,2019-12-30T21,ERCO - Coast,10361,megawatthours
4,2019-12-30T20,ERCO - Coast,10483,megawatthours


# Weather data collection

In [4]:
def split_dates_monthly(start_date, end_date):
    start = datetime.strptime(start_date, "%d-%m-%Y")
    end = datetime.strptime(end_date, "%d-%m-%Y")
    
    date_ranges = []
    current_date = start
    
    while current_date <= end:
        next_month = current_date + relativedelta(months=1)
        last_day_of_month = next_month - relativedelta(days=1)
        
        end_of_range = min(last_day_of_month, end)
        date_ranges.append([current_date.strftime("%Y-%m-%d"), end_of_range.strftime("%Y-%m-%d")])
        
        current_date = next_month
    
    return date_ranges

def get_weather_data(location, start_date, end_date):
    print("Getting weather data")
    monthly_date_ranges = split_dates_monthly(start_date, end_date)
    
    all_dataframes = []

    for start, end in monthly_date_ranges:    
        print("Getting data for dates", start, end)

        weather_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={0}&q={1}&format=json&date={2}&enddate={3}&tp=1".format(WEATHER_API_KEY, location, start, end)
        response = requests.get(weather_url)
        if response.status_code == 200:
            json_data = response.json()
            df_weather = pd.DataFrame(json_data["data"]["weather"])
            all_dataframes.append(df_weather)
        else:
            print(f"Failed to retrieve data for {start} to {end}: {response.status_code}")
    
    # Combine all DataFrames into a single DataFrame
    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        return combined_df
    else:
        return ""
    
def process_weather_data(df_weather):
    processed_data = []
    for index, row in df_weather.iterrows():
        date = row["date"]
        hourly_data = row["hourly"]

        for hour in hourly_data:
            time = hour["time"]
            time = time.zfill(4)
            hour_of_day = time[:2]

            datetime_str = f"{date}T{hour_of_day}"
            
            hour["datetime"] = datetime_str
            processed_data.append(hour) 

    df_processed = pd.DataFrame(processed_data)
    cols = ['datetime'] + [col for col in df_processed.columns if col != 'datetime']
    df_processed = df_processed[cols]
    df_processed = df_processed.drop(columns=['time', 'tempC', 'windspeedKmph', 'weatherIconUrl', 'weatherDesc', 'winddirDegree', 'winddir16Point'])

    return df_processed
    
    
# start date and end date should be same month and year for this api
start_date = '01-01-2015'
end_date = '31-12-2015'
location = "30.2672,-97.7431" # austin

df_weather = get_weather_data(location, start_date, end_date)
df_processed = process_weather_data(df_weather)


     

Getting weather data
Getting data for dates 2015-01-01 2015-01-31
Getting data for dates 2015-02-01 2015-02-28
Getting data for dates 2015-03-01 2015-03-31
Getting data for dates 2015-04-01 2015-04-30
Getting data for dates 2015-05-01 2015-05-31
Getting data for dates 2015-06-01 2015-06-30
Getting data for dates 2015-07-01 2015-07-31
Getting data for dates 2015-08-01 2015-08-31
Getting data for dates 2015-09-01 2015-09-30
Getting data for dates 2015-10-01 2015-10-31
Getting data for dates 2015-11-01 2015-11-30
Getting data for dates 2015-12-01 2015-12-31


In [5]:
df_weather.head()

,date,astronomy,maxtempC,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex,hourly
0,2015-01-01,"[{'sunrise': '07:28 AM', 'sunset': '05:42 PM',...",5,41,2,36,3,37,0.0,1.0,1,"[{'time': '0', 'tempC': '2', 'tempF': '36', 'w..."
1,2015-01-02,"[{'sunrise': '07:28 AM', 'sunset': '05:43 PM',...",6,42,4,40,4,40,0.0,1.0,1,"[{'time': '0', 'tempC': '5', 'tempF': '41', 'w..."
2,2015-01-03,"[{'sunrise': '07:28 AM', 'sunset': '05:43 PM',...",14,57,3,37,7,45,0.0,10.0,2,"[{'time': '0', 'tempC': '4', 'tempF': '39', 'w..."
3,2015-01-04,"[{'sunrise': '07:28 AM', 'sunset': '05:44 PM',...",7,45,1,34,6,42,0.0,10.0,3,"[{'time': '0', 'tempC': '7', 'tempF': '45', 'w..."
4,2015-01-05,"[{'sunrise': '07:29 AM', 'sunset': '05:45 PM',...",11,51,-1,29,4,39,0.0,10.0,2,"[{'time': '0', 'tempC': '1', 'tempF': '33', 'w..."


In [6]:
df_processed.head()

,datetime,tempF,windspeedMiles,weatherCode,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,...,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex
0,2015-01-01T00,36,10,122,0.0,0.0,81,2,1,1029,...,36,-1,31,-2,28,14,23,-2,28,1
1,2015-01-01T01,36,9,122,0.0,0.0,82,2,1,1028,...,36,-1,31,-2,29,13,21,-2,29,1
2,2015-01-01T02,36,9,122,0.0,0.0,83,2,1,1027,...,36,-0,31,-2,29,12,20,-2,29,1
3,2015-01-01T03,36,8,122,0.0,0.0,83,2,1,1026,...,36,-0,31,-1,29,11,18,-1,29,1
4,2015-01-01T04,36,8,122,0.0,0.0,84,5,2,1026,...,36,-0,32,-1,30,11,17,-1,30,1


# Merging demand and weather

In [13]:
zones_texas = {
    "COAS": [29.749907, -95.358421],  # Houston
    "EAST": [32.351485, -95.301140],  # Tyler
    "FWES": [31.997345, -102.077915],  # Midland
    "NCEN": [32.78306, -96.80667],      # Dallas
    "NRTH": [33.913708, -98.493387],  # Wichita Falls
    "SCEN": [30.267153, -97.743057],  # Austin
    "SOUT": [26.203407, -98.230012],  # McAllen
    "WEST": [32.448736, -99.733144]   # Abilene
}

zones_new_england = {
    "4001": [43.661471, -70.255326],  # Portland, Maine
    "4002": [42.995640, -71.454789],  # Manchester, New Hampshire
    "4003": [44.475882, -73.212072],  # Burlington, Vermont
    "4004": [41.763710, -72.685097],  # Hartford, Connecticut
    "4005": [41.823989, -71.412834],  # Providence, Rhode Island
    "4006": [41.635693, -70.933777],  # New Bedford, Massachusetts (Southeast)
    "4007": [42.101483, -72.589811],  # Springfield, Massachusetts (Western/Central)
    "4008": [42.358894, -71.056742]   # Boston, Massachusetts (Northeast)
}

In [14]:
from datetime import datetime

def split_dates_yearwise(start_date, end_date):
    start_year = datetime.strptime(start_date, "%d-%m-%Y").year
    end_year = datetime.strptime(end_date, "%d-%m-%Y").year
    
    date_ranges = []
    
    for year in range(start_year, end_year + 1):
        if year == start_year:
            year_start = start_date  # Use the specified start date for the first year
        else:
            year_start = f"01-01-{year}"  # First day of the year for subsequent years
            
        year_end = f"31-12-{year}" if year < end_year else end_date  # End of the year or specified end_date
        date_ranges.append([year_start, year_end])
    
    return date_ranges

def generate_dataset(zones, start_date, end_data):
    api_calls = 0
    dates = split_dates_yearwise(start_date, end_data)
    df_map = {}

    for zone in zones:
        if api_calls >=500:
            break
        
        print(zone)
        zone_name = zone
        date_df_list = []

        for date in dates:
            start = date[0]
            end = date[1] 
            print(start, end)

            df_demand = get_demand_data(zone, start, end)

            if type(df_demand) == str:
                print("Demand data not fetched for" + zone + " " + start + " " + end)
                continue
            
            city_location = ','.join(map(str, zones[zone]))
            df_weather = get_weather_data(city_location, start, end)
            
            if type(df_weather) == str:
                print("Weather data not fetched for" + zone + " " + start + " " + end)
                continue
            
            df_weather = process_weather_data(df_weather)
            
            df_weather.rename(columns={'datetime': 'datetime'}, inplace=True)
            df_demand.rename(columns={'period': 'datetime'}, inplace=True)

            df_merged_dataset = pd.merge(df_weather, df_demand, on='datetime', how='inner')
            df_merged_dataset['zone'] = zone_name

            date_df_list.append(df_merged_dataset)
            
            api_calls +=12
        
        combined_df = pd.concat(date_df_list, ignore_index=True)
        df_map[zone] = combined_df
    return df_map

start_date = "01-01-2019"
end_data = "31-12-2023"

df_map = generate_dataset(zones_texas, start_date, end_data)





4007
01-01-2019 31-12-2019
Getting demand data
01-01-2019 31-12-2019
Getting weather data
Getting data for dates 2019-01-01 2019-01-31
Getting data for dates 2019-02-01 2019-02-28
Getting data for dates 2019-03-01 2019-03-31
Getting data for dates 2019-04-01 2019-04-30
Getting data for dates 2019-05-01 2019-05-31
Getting data for dates 2019-06-01 2019-06-30
Getting data for dates 2019-07-01 2019-07-31
Getting data for dates 2019-08-01 2019-08-31
Getting data for dates 2019-09-01 2019-09-30
Getting data for dates 2019-10-01 2019-10-31
Getting data for dates 2019-11-01 2019-11-30
Getting data for dates 2019-12-01 2019-12-31
01-01-2020 31-12-2020
Getting demand data
01-01-2020 31-12-2020
Getting weather data
Getting data for dates 2020-01-01 2020-01-31
Getting data for dates 2020-02-01 2020-02-29
Getting data for dates 2020-03-01 2020-03-31
Getting data for dates 2020-04-01 2020-04-30
Getting data for dates 2020-05-01 2020-05-31
Getting data for dates 2020-06-01 2020-06-30
Getting data fo

In [15]:
df_map.keys()

dict_keys(['4007', '4008'])

In [18]:
df_map["NRTH"].head()

,datetime,tempF,windspeedMiles,weatherCode,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,...,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex,subba-name,value,value-units,zone
0,2019-06-05T17,84,6,116,0.0,0.0,62,10,6,1008,...,84,13,20,31,89,7,ERCO - North,1175,megawatthours,NRTH
1,2019-06-05T18,82,7,176,0.1,0.0,62,10,6,1008,...,82,14,23,30,87,1,ERCO - North,1225,megawatthours,NRTH
2,2019-06-05T19,80,11,116,0.0,0.0,71,9,5,1008,...,80,16,25,29,84,1,ERCO - North,1262,megawatthours,NRTH
3,2019-06-05T20,77,8,113,0.0,0.0,79,8,4,1008,...,77,17,27,27,80,1,ERCO - North,1266,megawatthours,NRTH
4,2019-06-05T21,75,7,113,0.0,0.0,87,7,4,1008,...,75,18,30,26,79,1,ERCO - North,1245,megawatthours,NRTH


In [16]:
df_combined = pd.concat(df_map.values(), ignore_index=True)
df_combined.sort_values(by='datetime', inplace=True)
df_combined.to_csv('data/merged_zones_weather_demand_data_NE.csv', index=False)

In [12]:
import os

# merge all csv
folder_path = './data'
output_file = 'all_merged_zones_weather_demand_data.csv'

# List to store individual dataframes
dfs = []

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Load each CSV into a dataframe
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

# Concatenate all dataframes in the list into one dataframe
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged dataframe to a new CSV
merged_df.to_csv(output_file, index=False)

C:\Users\rajat\AppData\Local\Temp\ipykernel_13300\2502220989.py:15: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
